In [2]:
from flask import Flask, request, redirect, jsonify
from flask_cors import CORS
import json
import cv2
import mediapipe as mp
import pickle
import math
import os
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# media pipe pose 전역변수로 설정
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True,min_detection_confidence=0.3, model_complexity=0)
mp_drawing = mp.solutions.drawing_utils

In [5]:
# 포즈 감지 함수
def detectPose(image,pose,display=True):
    # input 이미지 복사
    output_image = image.copy()
    # RGB로 변환
    image_BGR = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    # 렌드마크 찍기
    image_BGR.flags.writeable = False
    results = pose.process(image_BGR)
    image_BGR.flags.writeable = True
    
    # 사진 사이즈 구하기
    height, width, _ = image.shape
    # 렌드마크 값 리스트 만들기
    landmarks = []
    visible = []
    if results.pose_landmarks:
        # 렌드마크 그리기
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks, connections=mp_pose.POSE_CONNECTIONS)
        #렌드마크 정규값 landmarks 리스트에 넣기
        for landmark in results.pose_landmarks.landmark:
            landmarks.append((int(landmark.x*width), int(landmark.y*height), (landmark.z*width)))
            # visible 값 따로 리스트에 넣기 
            visible.append(landmark.visibility)
    # 사진 결과 보기
    if display:
        plt.figure(figsize=[22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        # 3차원으로도 봐보기
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    else:
        
        return output_image, landmarks , visible

In [6]:
# 각도 구하는 함수
def calculateAngle(landmark1, landmark2, landmark3):
    x1,y1,_ = landmark1 # z 좌표는 사용 x
    x2,y2,_ = landmark2
    x3,y3,_ = landmark3
    # 3점의 각도 -> 선 2개 -> 각도
    angle = math.degrees(math.atan2(y3-y2, x3-x2) - math.atan2(y1-y2,x1-x2))

    angle = np.abs(angle)
    # 음수의 각도가 나오면 양수로 바꾸기
    if angle >180.0:
        angle = 360 - angle
        
    return angle

In [7]:
# 중앙 각도 구하는 함수
def center_Landmark(landmarks1,landmarks2):
    x1,y1,_ = landmarks1
    x2,y2,_ = landmarks2
    x_cen = (x1+x2)/2
    y_cen = (y1+y2)/2
    return x_cen,y_cen,_

In [8]:
app = Flask(__name__)
@app.route("/W_result", methods=['GET', 'POST'])
def W_result():
    if request.method == 'POST':
        # 값 받아오는 부분
        get_json = request.form['test']
        # 모델에 학습하고
        # 테스트 할 영상 경로 넣으면 됨
        video = cv2.VideoCapture('./test_video2.mp4')
        df = pd.DataFrame() # predict에 넣어 줄 X_test
        actions = ['bend', 'headup', 'right', 'waistup']

        while video.isOpened():
            # Read a frame.
            ok, frame = video.read()
            # Check if frame is not read properly.
            if not ok:
                # Break the loop.
                break
            # display -> False 넣어서 img랑 렌드마크 받기
            output_img, landmarks,visible = detectPose(frame,pose,display=False)

            if landmarks:
                # visible 가져오기
                left_shoulder = visible[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                right_shoulder = visible[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                left_hip = visible[mp_pose.PoseLandmark.LEFT_HIP.value]
                right_hip = visible[mp_pose.PoseLandmark.RIGHT_HIP.value]
                left_knee = visible[mp_pose.PoseLandmark.LEFT_KNEE.value]
                right_knee = visible[mp_pose.PoseLandmark.RIGHT_KNEE.value]
                left_ankle = visible[mp_pose.PoseLandmark.LEFT_ANKLE.value]
                right_ankle = visible[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
                right_pinky = visible[mp_pose.PoseLandmark.RIGHT_PINKY.value]
                left_pinkey = visible[mp_pose.PoseLandmark.LEFT_PINKY.value]
                # 어깨 - 엉덩이 - 무릎 => 올리는 동작 각도 구하기 위함 -> Main angle
                if left_hip>right_hip:
                    main_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])
                else:
                    main_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value])                                                    
                
                # 엉덩이 - 무릎 - 발목 => 무릎 꺾였는지 보기 위함
                if left_knee > right_knee:
                    knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                                    landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
                else:
                    knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                                    landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                                    landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
                                                    
                # 어깨 허리  중앙값
                cen_shoulder = center_Landmark(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value])
                cen_hip = center_Landmark(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value])
            
                # 코 - 어깨 - 엉덩이 => 목 각도
                head_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.NOSE.value],
                                            cen_shoulder,cen_hip)

                # 잘보이는 새끼손가락의 y 좌표
                if right_pinky > left_pinkey:
                    y_pinky = landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value][1]
                else:
                    y_pinky = landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value][1]

                # 엉덩이 중앙의 y좌표와 새끼손가락의 y 좌표사이의 거리 
                waist_len = cen_hip[1]-cen_shoulder[1]

                globals()['list'] = knee_angle,head_angle,waist_len
                df = df.append([globals()[f'list']])
                print(globals()['list'])
                loaded_model = pickle.load(open('03.Knn_model', 'rb'))
                result = loaded_model.predict_proba([globals()['list']])
                print(result)
                # 각 회마다 맞는 자세인지 매핑하기
                if result.argmax() ==0:
                    cv2.putText(output_img, actions[0], (120,200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                elif result.argmax() ==1:
                    cv2.putText(output_img, actions[1], (120,200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                elif result.argmax() == 2:
                    cv2.putText(output_img, actions[2], (120,200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                elif result.argmax() ==3:
                    cv2.putText(output_img, actions[3], (120,200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                
            cv2.imshow('img',output_img)
            cv2.waitKey(10)
        video.release()
        cv2.destroyAllWindows()

        # 제이슨 어레이에 담아서
        st = result
        json_test = json.loads(result)
        print(json_test)
        return json_test



if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://210.223.239.172:5000/ (Press CTRL+C to quit)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_6176\3034425392.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'list']])


(173.2679012931122, 145.58930330886233, -18.0)


C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\smhrd\AppData\Local\Temp\ipykernel_6176\3034425392.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'list']])
C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


[[0. 0. 1. 0.]]
(173.50752747109385, 145.53617337282523, -17.5)
[[0. 0. 1. 0.]]


...

(144.80643686629273, 148.08304566274148, -18.5)
[[0. 0. 1. 0.]]
(144.0666981646755, 146.1248965517411, -22.0)
[[0. 0. 1. 0.]]
(157.59026237794743, 149.55391726191039, -12.0)
[[0. 0. 1. 0.]]
(153.81712734275843, 148.57056206475428, -14.5)
[[0. 0. 1. 0.]]


C:\Users\smhrd\AppData\Local\Temp\ipykernel_6176\3034425392.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'list']])
C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\smhrd\AppData\Local\Temp\ipykernel_6176\3034425392.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'list']])
C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\smhrd\AppData\Local\Temp\ipykernel_6176\3034425392.py:75: FutureWarning: The frame.append method is depre

(150.054289181184, 148.2875016563311, -18.0)
[[0. 0. 1. 0.]]
(149.1643115375988, 146.37332476694186, -23.5)
[[0. 0. 1. 0.]]
(154.4170730752673, 144.06998914417343, -27.0)
[[0. 0. 1. 0.]]


C:\Users\smhrd\AppData\Local\Temp\ipykernel_6176\3034425392.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'list']])
C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\smhrd\AppData\Local\Temp\ipykernel_6176\3034425392.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'list']])
C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\smhrd\AppData\Local\Temp\ipykernel_6176\3034425392.py:75: FutureWarning: The frame.append method is depre

(168.66041132805202, 144.3922211627145, -32.0)
[[0. 0. 1. 0.]]
(159.36214220572188, 144.52140416595898, -36.0)
[[0. 0. 1. 0.]]
(167.53870943078192, 143.3584348901375, -43.5)
[[0. 0. 1. 0.]]
(166.10080409459022, 136.64436864730703, -57.0)
[[0. 0. 1. 0.]]
